In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
from autogen_core.models import UserMessage, AssistantMessage, SystemMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from datetime import datetime

with open('gemini_key.txt', 'r') as f:
    gemini_key = f.read().strip()

gemini_llm_config = {
    "config_list": [
        {
            "model": "gemini-1.5-flash-8b", 
            "api_key": gemini_key,  
            "api_type": "google",
        }
    ],
    "cache_seed": None,  
}

In [12]:
class MockDatabase:
    def __init__(self):
        self._storage = ["ex1", "ex2", "ex3"]

    def save(self, value: str) -> str:
        self._storage.append(value)
        return f"Data saved: {value}"
        
    def get_all(self) -> list:
        return self._storage

mock_db = MockDatabase()

def save_data(value: str) -> str:
    return mock_db.save(value)

def show_stored_data() -> list:
    return mock_db.get_all()

from autogen import ConversableAgent
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can save user input and show previously stored data.",
    llm_config=gemini_llm_config,
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="save_data", description="Save user input to mock database")(save_data)
assistant.register_for_llm(name="show_stored_data", description="Show all stored data")(show_stored_data)
user_proxy.register_for_execution(name="save_data")(save_data)
user_proxy.register_for_execution(name="show_stored_data")(show_stored_data)

from autogen import register_function

register_function(
    save_data,
    caller=assistant,
    executor=user_proxy,
    name="save_data",
    description="Save user input to mock database",
)

register_function(
    show_stored_data,
    caller=assistant,  
    executor=user_proxy,  
    name="show_stored_data",  
    description="Show all saved data from mock database",  
)
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Save user input to mock database',
   'name': 'save_data',
   'parameters': {'type': 'object',
    'properties': {'value': {'type': 'string', 'description': 'value'}},
    'required': ['value']}}},
 {'type': 'function',
  'function': {'description': 'Show all saved data from mock database',
   'name': 'show_stored_data',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}}]

In [13]:
chat_result = user_proxy.initiate_chat(assistant,message="show me all stored data.")

User (to Assistant):

show me all stored data.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):


***** Suggested tool call (3094): show_stored_data *****
Arguments: 
{}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION show_stored_data...
Call ID: 3094
Input arguments: {}

>>>>>>>> EXECUTED FUNCTION show_stored_data...
Call ID: 3094
Input arguments: {}
Output:
['ex1', 'ex2', 'ex3']
User (to Assistant):

***** Response from calling tool (3094) *****
['ex1', 'ex2', 'ex3']
*********************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

OK.  What would you like me to do next?


--------------------------------------------------------------------------------
User (to Assistant)

ServerError: 500 INTERNAL. {'error': {'code': 500, 'message': 'Internal error encountered.', 'status': 'INTERNAL'}}

In [ ]:
class MockDatabase:
    def __init__(self):
        self._storage = ["ex1", "ex2", "ex3"]

    def save(self, value: str) -> str:
        self._storage.append(value)
        return f"Data saved: {value}"
        
    def get_all(self) -> list:
        return self._storage

mock_db = MockDatabase()

async def save_data(value: str) -> str:
    return mock_db.save(value)

async def get_data() -> list:
    return mock_db.get_all()

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=gemini_key,
)

agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[save_data, get_data],
    system_message="You are a professional assistant. Use the provided tools to save and return user data",
)

In [3]:
result = await agent.run(task="save 'ex5' and then get all saved data")
print(result.messages)
print(result.messages[-1].content)

[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 18, 19, 10, 8, 765958, tzinfo=datetime.timezone.utc), content="save 'ex5' and then get all saved data", type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=9), metadata={}, created_at=datetime.datetime(2025, 6, 18, 19, 10, 9, 613337, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='', arguments='{"value":"ex5"}', name='save_data'), FunctionCall(id='', arguments='{}', name='get_data')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 18, 19, 10, 9, 613337, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Data saved: ex5', name='save_data', call_id='', is_error=False), FunctionExecutionResult(content="['ex1', 'ex2', 'ex3', 'ex5']", name='get_data', call_id='', is_error=False)], type='ToolCallExec